## 2019년 부터 2023년까지 전체 기사에 대한 LDA 모델 적용

In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import os
from gensim import corpora
from gensim.models import LdaModel, TfidfModel

In [3]:
# 합칠 csv파일 확인
file_list = os.listdir('/content/drive/MyDrive/2023년 10월-11월 프로젝트/토큰화 진행 및 불용어 처리/')
file_list_csv = [file for file in file_list if file.endswith('.csv')]
print('대상 파일 리스트: {}'.format(file_list_csv))

대상 파일 리스트: ['2019년 기사(완료).csv', '2020년 기사(완료).csv', '2021년 기사(완료).csv', '2023년 기사(완료).csv', '2022년 기사(완료).csv']


In [4]:
# csv 합치기
article_data = {}
for article in file_list_csv:
  article_data[article] = pd.read_csv('/content/drive/MyDrive/2023년 10월-11월 프로젝트/토큰화 진행 및 불용어 처리/{}'.format(article))

df = pd.concat(article_data.values(), ignore_index=True)
df.drop(columns=['Unnamed: 0','title','content','tokenize_title'],inplace=True) # LDA대상은 기사내용(content)만으로 함
df.reset_index(drop=True, inplace=True)
df

,date,url,category,tokenize_content
0,2019-01-01T07:31:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['블록', '체인', '시장', '진단', '한다', '사실', '앞', '겸손한..."
1,2019-01-01T13:15:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"['종목', '상담', '카카오', '방송', '된', '국민', '주식', '고충..."
2,2019-01-02T04:03:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"['재계', '뛴다', '카카오', '블록', '체인', '기술', '열사', '라..."
3,2019-01-02T06:05:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['카카오', '카카오', '프렌즈', '앞세운', '캐릭터', '사업', '본격'..."
4,2019-01-02T09:41:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['배우', '매니지먼트', '개사', '광고', '모델', '캐스팅', '에이전시..."
...,...,...,...,...
32174,2022-12-30T12:20:00,https://n.news.naver.com/mnews/article/031/000...,IT/과학,"['지난', '간', '이동', '트렌드', '분석', '해외', '여행', '수요..."
32175,2022-12-30T13:04:00,https://n.news.naver.com/mnews/article/119/000...,IT/과학,"['카카오', '피해', '보상', '발표', '소상', '공인', '피해', '지..."
32176,2022-12-30T16:15:00,https://n.news.naver.com/mnews/article/015/000...,사회,"['서랍', '플러스', '이용권', '보상', '지급', '전체', '이용자', ..."
32177,2022-12-30T17:07:00,https://n.news.naver.com/mnews/article/138/000...,IT/과학,"['카카오', '지난', '서비스', '장애', '발생', '이후', '보상', '..."


In [5]:
# 'tokenize_content'의 type이 str이므로 이를 list로 변환하기 위함
def str_list(token):
  str_list = token
  str_list = str_list.replace("[","").replace("]","").replace("'","").replace(" ","")
  result = str_list.split(",")
  return result

In [6]:
# content에 대한 LDA모델 적용
content_docs = []
for doc in df['tokenize_content']:
  content_docs.append(str_list(doc))
content_docs

Output hidden; open in https://colab.research.google.com to view.

In [7]:
id2word_content = corpora.Dictionary(content_docs)
print(id2word_content)

Dictionary<76512 unique tokens: ['가늠', '가능성', '가능하도록', '가이드라인', '가입자']...>


In [8]:
for value in id2word_content:
  print(value, id2word_content[value])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
71512 꺼려하지만
71513 박홍
71514 정몽혁
71515 진종
71516 롱코
71517 배출권거래제
71518 슈어런
71519 대쉬보드
71520 숨쉬기
71521 계략
71522 골칫
71523 황재헌
71524 강원대학교
71525 랜들
71526 임함
71527 쉬자
71528 걷히
71529 도정
71530 비엠아이
71531 오설록
71532 지에스건설
71533 확득
71534 기다려주시는
71535 케해
71536 환혼
71537 보섬
71538 같아지도록
71539 어슈어런스
71540 에몬스
71541 에몬스홈
71542 프레디
71543 싱크로나이즈
71544 앉아있어도
71545 틸팅락
71546 지웠다
71547 후려쳐야
71548 궁금해졌다는
71549 단단해졌을
71550 오두막
71551 외로웠던
71552 지내면
71553 쫓겨서
71554 채워지니까
71555 매달아서
71556 주렁주렁
71557 않았나라는
71558 외로웠었다
71559 지난날
71560 키워주며
71561 걸작
71562 장웅
71563 다졌다며
71564 김정현
71565 위에브
71566 나가는데도
71567 박광식
71568 압구정로
71569 언주로
71570 영동대로
71571 납금
71572 리세
71573 가있어하나
71574 김성아
71575 까먹는다
71576 나타나서
71577 냅다
71578 노션
71579 뛰어다니면서
71580 뛰어주실
71581 만만하지만
71582 모여있을
71583 물렁하다
71584 박만
71585 받았다던지
71586 받으셔야
71587 배움이었다
71588 배웠는지도
71589 불합격
71590 비오는
71591 비저닝
71592 송경찬
71593 쌓아가면서
71594 써놓는다거나
71595 써야하는
71596 앉아있어야
71597 어땠다는
71598 어려워하시는
71599 유기체
71600 윤예
71601 이연주
71602 저릴
7

In [9]:
corpus_TDM = []
for doc in content_docs:
  result = id2word_content.doc2bow(doc)
  corpus_TDM.append(result)

corpus_TDM # index, 등장 횟수 (순서는 index 순으로 sorting해서 출력됨)

Output hidden; open in https://colab.research.google.com to view.

In [10]:
# 토큰화 진행 및 불용어 처리에서 불용어에 대해 업데이트 해줘야함!!!
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 50 # 토픽갯수 - 변경가능
lda = LdaModel(corpus=corpus_TFIDF, id2word=id2word_content, num_topics=n, random_state=100)

for t in lda.print_topics():
  print(t[0],":",t[1])

35 : 0.059*"시세" + 0.026*"압수수색" + 0.020*"비트" + 0.004*"클레이" + 0.004*"코인" + 0.003*"화폐" + 0.002*"저축은행" + 0.002*"퍼포먼스" + 0.001*"암호" + 0.001*"나무"
32 : 0.012*"사외이사" + 0.005*"정관" + 0.003*"원안" + 0.003*"재무제표" + 0.002*"정기" + 0.002*"신한은행" + 0.001*"쏠" + 0.000*"규진" + 0.000*"환승" + 0.000*"신인"
8 : 0.035*"방통위" + 0.007*"알뜰폰" + 0.002*"금지법" + 0.002*"이랜드" + 0.002*"연금" + 0.001*"중소기업" + 0.001*"공제" + 0.000*"선착순" + 0.000*"세액" + 0.000*"멤버십"
29 : 0.018*"국정감사" + 0.017*"법안" + 0.014*"지침" + 0.014*"외국인" + 0.013*"매도" + 0.011*"상위" + 0.011*"종목" + 0.009*"기관" + 0.009*"해제" + 0.008*"전자"
36 : 0.004*"콘텐트" + 0.000*"웍" + 0.000*"스모" + 0.000*"바일" + 0.000*"오디션" + 0.000*"유튜브" + 0.000*"사진전" + 0.000*"웍스" + 0.000*"썼다" + 0.000*"번역"
28 : 0.066*"사과" + 0.029*"지수" + 0.017*"나스닥" + 0.016*"마감" + 0.015*"금리" + 0.013*"인상" + 0.012*"포인트" + 0.011*"물가" + 0.011*"형제" + 0.009*"준"
26 : 0.092*"대구" + 0.037*"대구시" + 0.009*"선물" + 0.008*"쇼핑" + 0.008*"과일" + 0.007*"식품" + 0.007*"할인" + 0.005*"기획전" + 0.005*"선착순" + 0.004*"세트"
30 : 0.027*"사주" + 0.015*"예수" + 0.002*"카모

In [11]:
# 토픽배정
for i in range(len(corpus_TFIDF)):
  dominant_topic = sorted(lda[corpus_TFIDF[i]], key=lambda x: x[1], reverse=True)[0]
  df.loc[i, 'topic(content)'] = dominant_topic[0]
  df.loc[i, 'topic_details(content)'] = dominant_topic[1]
df

,date,url,category,tokenize_content,topic(content),topic_details(content)
0,2019-01-01T07:31:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['블록', '체인', '시장', '진단', '한다', '사실', '앞', '겸손한...",44.0,0.552564
1,2019-01-01T13:15:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"['종목', '상담', '카카오', '방송', '된', '국민', '주식', '고충...",44.0,0.522721
2,2019-01-02T04:03:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"['재계', '뛴다', '카카오', '블록', '체인', '기술', '열사', '라...",44.0,0.488279
3,2019-01-02T06:05:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['카카오', '카카오', '프렌즈', '앞세운', '캐릭터', '사업', '본격'...",44.0,0.678018
4,2019-01-02T09:41:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['배우', '매니지먼트', '개사', '광고', '모델', '캐스팅', '에이전시...",44.0,0.370060
...,...,...,...,...,...,...
32174,2022-12-30T12:20:00,https://n.news.naver.com/mnews/article/031/000...,IT/과학,"['지난', '간', '이동', '트렌드', '분석', '해외', '여행', '수요...",44.0,0.464399
32175,2022-12-30T13:04:00,https://n.news.naver.com/mnews/article/119/000...,IT/과학,"['카카오', '피해', '보상', '발표', '소상', '공인', '피해', '지...",44.0,0.587038
32176,2022-12-30T16:15:00,https://n.news.naver.com/mnews/article/015/000...,사회,"['서랍', '플러스', '이용권', '보상', '지급', '전체', '이용자', ...",44.0,0.573906
32177,2022-12-30T17:07:00,https://n.news.naver.com/mnews/article/138/000...,IT/과학,"['카카오', '지난', '서비스', '장애', '발생', '이후', '보상', '...",44.0,0.418469


In [12]:
# 토픽별 비중 확인
df['topic(content)'].unique()

array([44., 33., 47., 15.,  9., 17., 41.,  1., 18., 29., 28., 26., 38.,
       25., 12.,  5., 39., 21.,  0.,  2., 24., 45.])

In [13]:
df['topic(content)'].value_counts()

44.0    22668
15.0     3241
33.0     2339
41.0     2085
17.0      944
18.0      348
9.0       124
29.0       93
47.0       80
38.0       72
1.0        55
28.0       36
39.0       25
0.0        22
25.0       13
12.0       11
2.0        11
26.0        7
24.0        2
21.0        1
5.0         1
45.0        1
Name: topic(content), dtype: int64

In [14]:
# CSV 파일로 저장
csv_filename = "/content/drive/MyDrive/2023년 10월-11월 프로젝트/LDA모델 적용/news_전체.csv"
df.to_csv(csv_filename, index=False)

## 시각화

In [15]:
!pip install pyLDAvis==3.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 26.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicor

In [16]:
!pip install pandas==1.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 20.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.2
    Uninstalling pandas-2.1.2:
      Successfully uninstalled pandas-2.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 1.5.1 which is incompatible.
pyldavis 3.4.1 requires pandas>=2.0.0, but you have pandas 1.5.1 which is incompatible.


In [17]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus_TFIDF, id2word_content)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
